# <span style='color:blue'> RBDO with OpenTURNS </span>

RBDO class is used to solve such reliability-based design optimization problem:

\begin{eqnarray}
\min_{\mathbf{d},\mathbf{p}} & & C(\mathbf{d},\mathbf{p}) \\
\text{s.t.} & & \left\{
                \begin{array}{ll}
                  \mathbb{P} \left(g_i(\mathbf{d},\mathbf{X}(\mathbf{p}),\mathbf{Z})\leq 0 \right) \leq P_{f_i}^T, \;\; i= 1,\ldots,m\\
                  h_j(\mathbf{d},\mathbf{p})\leq 0, \;\; j= 1,\ldots,M\\
                \end{array}
              \right.
\end{eqnarray}
with $\mathbf{d} \in \mathbb{R}^{n_d} $, $\mathbf{p}\in\mathbb{R}^{n_p}$, $\mathbf{X}\sim \mathbf{f}_{\mathbf{X}|\mathbf{d}}(\cdot)$ of dimension $n_X$ and $\mathbf{Z}\sim \mathbf{f}_{\mathbf{Z}}(\cdot)$

- $\mathbf{d}$ is a vector of deterministic variables
- $\mathbf{p}$ is a vector of deterministic variables corresponding to hyperparameters of the random variables $\mathbf{X}$ defined by its joint PDF $\mathbf{f}_{\mathbf{X}|\mathbf{d}}(\cdot)$ (e.g., mean, standard deviation, shape parameters) 
- $\mathbf{X}$ is a random vector with some hyperparameters defining its PDF that can depend on $\mathbf{p}$
- $\mathbf{Z}$ is a random vector with fixed hyperparameters defined by the PDF $\mathbf{f}_{\mathbf{Z}}(\cdot)$
- $P_{f_i}^T$ is a threshold of maximal probability value for the constraint $i$


Three algorithms are currently available in RBDO class to solve such a problem:
- <span style='color:blue'> RIA </span>  (Reliability Index Approach) algorithm
- <span style='color:blue'> PMA </span> (Performance Measure Approach) algorithm with 4 different methods for inverse FORM: 'SLSQP', 'AMV', 'CMV' and 'HMV'
- <span style='color:blue'> SORA </span> (Sequential Optimization and Reliability Assessment) algorithm with 4 different methods for inverse FORM: 'SLSQP', 'AMV', 'CMV' and 'HMV'

Please refer to the following papers for more details on the algorithms:
- Valdebenito, Marcos A., and Gerhart I. Schuëller. "A survey on approaches for reliability-based optimization." Structural and Multidisciplinary Optimization 42.5 (2010): 645-663.
- Aoues, Younes, and Alaa Chateauneuf. "Benchmark study of numerical methods for reliability-based design optimization." Structural and multidisciplinary optimization 41.2 (2010): 277-294.
- Du, Xiaoping, and Wei Chen. "Sequential optimization and reliability assessment method for efficient probabilistic design." J. Mech. Des. 126.2 (2004): 225-233.

## Analytical example 

To illustrate the RBDO class, let's consider the following classical RBDO problem [see the paper Aoues, Younes, and Alaa Chateauneuf (2010)] 

\begin{eqnarray}
\min_{p_1,p_2} & & p_1+p_2 \\
\text{s.t.} & & \left\{
                \begin{array}{ll}
                  \mathbb{P} \left(g_i(\mathbf{X}(p_1,p_2))\leq 0 \right) \leq P_{f_i}^T, \;\; i= 1,\ldots,3\\
                  0 \leq p_1\leq 10 \\
                  0 \leq p_2\leq 10 \\
                \end{array}
              \right.
\end{eqnarray}
with:
- $g_1(\mathbf{X}) = X_1^2X_2/20-1$ 
- $g_2(\mathbf{X}) = (X_1+X_2-5)^2/30+(X_1-X_2-12)^2/120-1$
- $g_3(\mathbf{X}) = 80/(X_1^2+8X_2+5)-1$

and $\mathbf{X}=[X_1,X_2]$ a random vector of two independant Gaussian variables $X_1\sim \mathcal{N}(p_1,0.6),X_2\sim \mathcal{N}(p_2,0.6)$, $[p_1,p_2] = [\mu_{X_1},\mu_{X_2}]$ the mean values of the random variables $X_1,X_2$.
For the probability target maximal, it is assumed a reliability index $\beta_i = 2.$ leading to $P_{f_i}^T = \Phi(-\beta_i) \simeq 0.02275$ with $\Phi(\cdot)$ the CFD of a standard Normal distribution.

In [1]:
import openturns as ot
from RBDO_class import *
import time

In [2]:
## Definition of the objective Python function
def obj(d,p):
    p0 = p[0]
    p1 = p[1]        
    return p0+p1

## Definition of the objective RBDO function with active d and p on the objective function. 
## In the example, no d are given (no active d), and the objective function depends on p=[p_1,p_2] (both active on the function),
## therefore in the vector XX=[d,p] p_1,p_2 are first and second index of XX
active_index_d = []
active_index_p = [0,1]
len_XX = 2
f_obj = Objective(obj,active_index_d,active_index_p,len_XX)


## Definition of the constraint functions
def Constraint1_ori(d,x,z):
    x1 = x[0]
    x2 = x[1]
    g1 = x1**2*x2/20.-1. 
    return [g1]

def Constraint2_ori(d,x,z):
    x1 = x[0]
    x2 = x[1]
    g2 = (x1+x2-5.)**2/30.+(x1-x2-12.)**2/120.-1.
    return [g2]

def Constraint3_ori(d,x,z):
    x1 = x[0]
    x2 = x[1]
    g3 = 80./(x1**2+ 8.*x2+5.)-1. 
    return [g3]

## Definition of the parametric distributions for X for each constraint
def updatedistg1(p):
    
    m_1 = p[0]
    m_2 = p[1]
    marg1 = ot.Normal(m_1,0.6)
    marg2 = ot.Normal(m_2,0.6)
    dist = ot.ComposedDistribution([marg1,marg2])
    return dist

def updatedistg2(p):
    
    m_1 = p[0]
    m_2 = p[1]
    marg1 = ot.Normal(m_1,0.6)
    marg2 = ot.Normal(m_2,0.6)
    dist = ot.ComposedDistribution([marg1,marg2])
    return dist

def updatedistg3(p):
    
    m_1 = p[0]
    m_2 = p[1]
    marg1 = ot.Normal(m_1,0.6)
    marg2 = ot.Normal(m_2,0.6)
    dist = ot.ComposedDistribution([marg1,marg2])
    return dist


### RIA algorithm

In [3]:
## Definition of the RBDO probabilistic constraints 
active_index_d_g1 = [] ## active index of d for constraint g1
active_index_p_g1 = [0,1] ## active index of p for constraint g1
distZ_g1= [] ## distribution of Z for constraint g1
PfT_g1 = 0.02275 ## Probability maximal target for constraint g1
PIneq_g1 = PIneqCons(Constraint1_ori,active_index_d_g1,active_index_p_g1,updatedistg1,distZ_g1,PfT_g1)

active_index_d_g2 = [] ## active index of d for constraint g2
active_index_p_g2 = [0,1] ## active index of p for constraint g2
distZ_g2 = [] ## distribution of Z for constraint g2
PfT_g2 = 0.02275 ## Probability maximal target for constraint g2
PIneq_g2 = PIneqCons(Constraint2_ori,active_index_d_g2,active_index_p_g2,updatedistg2,distZ_g2,PfT_g2)

active_index_d_g3 = [] ## active index of d for constraint g3
active_index_p_g3 = [0,1] ## active index of p for constraint g3
distZ_g3 = [] ## distribution of Z for constraint g3
PfT_g3 = 0.02275 ## Probability maximal target for constraint g3
PIneq_g3 = PIneqCons(Constraint3_ori,active_index_d_g3,active_index_p_g3,updatedistg3,distZ_g3,PfT_g3)

## Solve of the problem using RIA algorithm

## Definition of the list of constraints
PIneqCons_list = [PIneq_g1,PIneq_g2,PIneq_g3]  #list of Probability inequality constraints -->g_1, g_2 and g_3 here
DIneqCons_list = [] #list of deterministic constraints --> no deterministic constraints here
Bounds = ot.Interval([0.,0.], [10.,10.])  #Definition of the bounds on XX=[d,p] 
len_d = 0 #len of deterministic variable vector d
len_p = 2 #len of hyperparemeters variable vector p

# Definition of RBDO problem using the previously defined objective function, constraint functions and bounds
RBDO_Problem_test = RBDO_Problem(f_obj,PIneqCons_list,DIneqCons_list,Bounds,len_d,len_p)

# Initial optimization points
InitialPoint = [5.0,5.0]

# Solver type of RIA
Solver = ot.NLopt('LN_COBYLA')

# Instantiation of the RIA algorithm
RIA_Algorithm_test = RIA_Algorithm(RBDO_Problem_test,Solver,InitialPoint)

t0 = time.time()
# Run of RIA algorithm
result = RIA_Algorithm_test.run()

#Obtained results
print('Optimal design variables p = [p_1,p_2] = ',RIA_Algorithm_test.get_optimum())
print('Optimal objective function value = ',RIA_Algorithm_test.get_foptimum())
print('Optimal constraint functions value = ',RIA_Algorithm_test.get_consoptimum())
print('Number of RIA iterations = ', result.getEvaluationNumber())
print('Time CPU = ',time.time()-t0)

Optimal design variables p = [p_1,p_2] =  [3.60893,3.65932]
Optimal objective function value =  [7.26825]
Optimal constraint functions value =  [0.022750654083177963, 0.02275081839303613, 4.589794583032902e-06]
Number of RIA iterations =  20
Time CPU =  0.30506086349487305


### PMA algorithm

In [4]:
## Solve of the problem using PMA algorithm

## Definition of the list of constraints for PMA
active_index_d_g1 = [] ## active index of d for constraint g1
active_index_p_g1 = [0,1] ## active index of p for constraint g1
distZ_g1= [] ## distribution of Z for constraint g1
PfT_g1 = 0.02275 ## Probability maximal target for constraint g1
InvFORM_solver_g1 = 'SLSQP' ## Choice of Inverse FORM Solver : ['SLSQP','AMV','CMV','HMV']
PIneq_g1 = PIneqCons(Constraint1_ori,active_index_d_g1,active_index_p_g1,updatedistg2,distZ_g1,PfT_g1,solver_invFORM = InvFORM_solver_g1) 

active_index_d_g2 = [] ## active index of d for constraint g2
active_index_p_g2 = [0,1] ## active index of p for constraint g2
distZ_g2 = [] ## distribution of Z for constraint g2
PfT_g2 = 0.02275 ## Probability maximal target for constraint g2
InvFORM_solver_g2 = 'SLSQP' ## Choice of Inverse FORM Solver : ['SLSQP','AMV','CMV','HMV']
PIneq_g2 = PIneqCons(Constraint2_ori,active_index_d_g2,active_index_p_g2,updatedistg2,distZ_g2,PfT_g2,solver_invFORM = InvFORM_solver_g2) 

active_index_d_g3 = [] ## active index of d for constraint g3
active_index_p_g3 = [0,1] ## active index of p for constraint g3
distZ_g3 = [] ## distribution of Z for constraint g3
PfT_g3 = 0.02275 ## Probability maximal target for constraint g3
InvFORM_solver_g3 = 'SLSQP' ## Choice of Inverse FORM Solver : ['SLSQP','AMV','CMV','HMV']
PIneq_g3 = PIneqCons(Constraint3_ori,active_index_d_g3,active_index_p_g3,updatedistg3,distZ_g3,PfT_g3,solver_invFORM = InvFORM_solver_g3) 

PIneqCons_list = [PIneq_g1,PIneq_g2,PIneq_g3]#list of Probability inequality constraints -->g_1, g_2 and g_3 here
DIneqCons_list = []#list of deterministic constraints --> no deterministic constraints here
Bounds = ot.Interval([0.,0.], [10.,10.] ) #Definition of the bounds on XX=[d,p] 
len_d = 0 #len of deterministic variable vector d
len_p = 2 #len of hyperparemeters variable vector p

# Definition of RBDO problem using the previously defined objective function, constraint functions and bounds
RBDO_Problem_test = RBDO_Problem(f_obj,PIneqCons_list,DIneqCons_list,Bounds,len_d,len_p)

# Initial optimization points
InitialPoint = [5.0,5.0] 

# Solver type of PMA
Solver = ot.NLopt('LN_COBYLA')

t0 = time.time()
# Instantiation of the PMA algorithm
PMA_Algorithm_test = PMA_Algorithm(RBDO_Problem_test,Solver,InitialPoint)
result = PMA_Algorithm_test.run()

#Obtained results
print('Optimal design variables p = [p_1,p_2] = ',PMA_Algorithm_test.get_optimum())
print('Optimal objective function value = ',PMA_Algorithm_test.get_foptimum())
print('Optimal constraint functions value = ',PMA_Algorithm_test.get_consoptimum())
print('Number of PMA iterations = ', result.getEvaluationNumber())
print('Time CPU = ',time.time()-t0)

Optimal design variables p = [p_1,p_2] =  [3.60894,3.65932]
Optimal objective function value =  [7.26826]
Optimal constraint functions value =  [0.022750000535502517, 0.022749999852842078, 0.010376551017194577]
Number of PMA iterations =  32
Time CPU =  0.31107234954833984


### SORA algorithm

In [5]:
## Definition of the list of constraints for SORA
active_index_d_g1 = [] ## active index of d for constraint g1
active_index_p_g1 = [0,1] ## active index of p for constraint g1
distZ_g1= [] ## distribution of Z for constraint g1
PfT_g1 = 0.02275 ## Probability maximal target for constraint g1
InvFORM_solver_g1 = 'SLSQP' ## Choice of Inverse FORM Solver : ['SLSQP','AMV','CMV','HMV']
PIneq_g1 = PIneqCons(Constraint1_ori,active_index_d_g1,active_index_p_g1,updatedistg2,distZ_g1,PfT_g1,solver_invFORM = InvFORM_solver_g1) 

active_index_d_g2 = [] ## active index of d for constraint g2
active_index_p_g2 = [0,1] ## active index of p for constraint g2
distZ_g2 = [] ## distribution of Z for constraint g2
PfT_g2 = 0.02275 ## Probability maximal target for constraint g2
InvFORM_solver_g2 = 'SLSQP' ## Choice of Inverse FORM Solver : ['SLSQP','AMV','CMV','HMV']
PIneq_g2 = PIneqCons(Constraint2_ori,active_index_d_g2,active_index_p_g2,updatedistg2,distZ_g2,PfT_g2,solver_invFORM = InvFORM_solver_g2) 

active_index_d_g3 = [] ## active index of d for constraint g3
active_index_p_g3 = [0,1] ## active index of p for constraint g3
distZ_g3 = [] ## distribution of Z for constraint g3
PfT_g3 = 0.02275 ## Probability maximal target for constraint g3
InvFORM_solver_g3 = 'SLSQP' ## Choice of Inverse FORM Solver : ['SLSQP','AMV','CMV','HMV']
PIneq_g3 = PIneqCons(Constraint3_ori,active_index_d_g3,active_index_p_g3,updatedistg3,distZ_g3,PfT_g3,solver_invFORM = InvFORM_solver_g3) 

PIneqCons_list = [PIneq_g1,PIneq_g2,PIneq_g3]#list of Probability inequality constraints -->g_1, g_2 and g_3 here
DIneqCons_list = []#list of deterministic constraints --> no deterministic constraints here
Bounds = ot.Interval([0.,0.], [10.,10.] ) #Definition of the bounds on XX=[d,p] 
len_d = 0 #len of deterministic variable vector d
len_p = 2 #len of hyperparemeters variable vector p

# Definition of RBDO problem using the previously defined objective function, constraint functions and bounds
RBDO_Problem_test = RBDO_Problem(f_obj,PIneqCons_list,DIneqCons_list,Bounds,len_d,len_p)

# Initial optimization points
InitialPoint = [5.0,5.0] 

# Solver type of SORA
Solver = ot.NLopt('LN_COBYLA')

t0 = time.time()
# Instantiation of the SORA algorithm
SORA_Algorithm_test = SORA_Algorithm(RBDO_Problem_test,Solver,InitialPoint)
result = SORA_Algorithm_test.run()

#Obtained results
print('Optimal design variables p = [p_1,p_2] = ',SORA_Algorithm_test.get_optimum())
print('Optimal constraint functions value = ',SORA_Algorithm_test.get_consoptimum())
print('Optimal objective function value = ',SORA_Algorithm_test.get_foptimum())
print('Number of SORA iterations = ', result.getEvaluationNumber())
print('Time CPU = ',time.time()-t0)

Optimal design variables p = [p_1,p_2] =  [3.60894,3.65932]
Optimal constraint functions value =  [0.022750002984634525, 0.02275000016160846, 0.01037655056765985]
Optimal objective function value =  [7.26826]
Number of SORA iterations =  28
Time CPU =  0.056627511978149414
